In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

#URL of the website
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

#Function to scrape Bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

#Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5) #Allow the page to load
    
        #Click the "View Buses" button if it exists
        try:
            view_buses_button =  WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'button'))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)

            #Scroll down to view all Bus details
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

           #Find Bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travelsName___950ec8")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bottomSection___95bf64")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "boardingTime___a78ae0")
            duration_elements = driver.find_elements(By.CLASS_NAME, "duration___b3a515")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "droppingTime___c814da")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'rating___f7ba20')]")
            price_elements = driver.find_elements(By.CLASS_NAME, "finalFare___057afc")

            # Use XPath to handle both seat availability classes 
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'totalSeats___53250b')]")
            
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details

        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

#Listing Bus details
all_bus_details = []

#Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  #There is 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                   EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load

            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
             # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

#Scrape routes and details from all pages
scrape_all_pages()

#Convert the list of dictionary into Dataframe
df = pd.DataFrame(all_bus_details)

#save Dataframe to CSV 
df.to_csv('Chandighar_Bus_details.csv', index=False)

#Close the driver
driver.quit()

NameError: name 'driver' is not defined

In [5]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi Holidays Pvt Ltd,,23:40,5h 5m,04:45,4.6,₹873,0
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,,04:50,4h 10m,09:00,4.6,₹425,0
2,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi Holidays Pvt Ltd,,03:20,3h 55m,07:15,4.6,₹417,0
3,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Khurana Bus Service,,21:50,5h 10m,03:00,3.7,₹339,0
4,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,NueGo,,20:00,5h 20m,01:20,4.3,₹589,0
...,...,...,...,...,...,...,...,...,...,...
89,Chandigarh to Dharamshala (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...,FlixBus,,02:20,6h 5m,08:25,4.5,₹735,0
90,Chandigarh to Dharamshala (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,,01:30,5h 55m,07:25,4.6,"₹1,249",0
91,Chandigarh to Dharamshala (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,,02:25,5h 20m,07:45,4.7,"₹1,386",0
92,Chandigarh to Dharamshala (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...,New Walia Travels,,01:20,5h 10m,06:30,5,₹998,0
